## Tensorflow for machine learning 
* training cnn (all code in separate py file)

In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import os
import itertools
import glob
%matplotlib inline

### Setup to read data from tfrecords file

In [ ]:
filename_queue = tf.train.string_input_producer(
    tf.train.match_filenames_once("./output/testing-images/*.tfrecords"))

reader = tf.TFRecordReader()
_, value = reader.read(filename_queue)

# features (X) dictionary
features = tf.parse_single_example(
    value,
    features={
        'label':tf.FixedLenFeature([], tf.string),
        'image':tf.FixedLenFeature([], tf.string)
    })

# definition of shape and data types
image = tf.reshape(tf.decode_raw(features['image'], tf.uint8), [250,151,1])
label = tf.cast(features['label'], tf.string)

In [ ]:
min_after_dequeue = 10
batch_size = 7
capacity = min_after_dequeue + 3 * batch_size + 10

image_batch, label_batch = tf.train.shuffle_batch(
    [image, label], batch_size=batch_size, capacity=capacity, min_after_dequeue=min_after_dequeue)
float_image_batch = tf.image.convert_image_dtype(image_batch, tf.float32)

### Setting up Network

In [ ]:
import glob
labels = map(lambda x: x.split("/")[-1], glob.glob('./imagenet-dogs/Images/*'))
#mapping = tf.map_fn(lambda l: tf.where(tf.equal(labels, l)[0][0]), label_batch, dtype=tf.float64)
m_ = tf.map_fn(lambda l: tf.cast(tf.where(tf.equal(labels,l)), dtype=tf.float32), label_batch, dtype=tf.float32)

In [ ]:
# imgs = []
# # labs = []
# sess = tf.Session()
    
# sess.run(tf.initialize_all_variables())
# coord = tf.train.Coordinator()
# threads = tf.train.start_queue_runners(coord=coord, sess=sess)

In [ ]:
sess.run([m_])[0].reshape(1,7)[0]

In [ ]:
coord.request_stop()
coord.join(threads)

### Building Network (Layers)

In [ ]:
conv_2d_layer_one = tf.contrib.layers.convolution2d(
    inputs=float_image_batch, 
    num_outputs=32,
    kernel_size=(5,5),
    activation_fn=tf.nn.relu,
    weights_initializer=tf.constant_initializer(0.0),
    stride=(2,2),
    trainable=True    
)

In [ ]:
sess = tf.Session()    
sess.run(tf.initialize_all_variables())
coord = tf.train.Coordinator()
threads = tf.train.start_queue_runners(coord=coord, sess=sess)
float_img_batch, conv_out = sess.run([float_image_batch, conv_2d_layer_one])

In [ ]:
# adding lables as one hot encoding